In [1]:
import os
import sys
import tqdm
import pandas as pd

In [2]:
REBADD_LIB_PATH = os.path.abspath(os.pardir)
if REBADD_LIB_PATH not in sys.path:
    sys.path = [REBADD_LIB_PATH] + sys.path

from rebadd.evaluate import evaluate_sr_nov_div

In [3]:
## Test data (navitoclax, ABT-737)
filepath_ref = os.path.join(os.pardir, 'data', 'zinc15', 'zinc15_test.txt')
df_ref = pd.read_csv(filepath_ref, header=None)

referece_smiles_iter = df_ref.iloc[:,0].values.tolist()

print(len(referece_smiles_iter))

2


In [4]:
input_dir = 'outputs_4_calculate_properties'

modelnames = [
    'zinc15',
]

filenames = [f'smi_after.csv.{num:04d}' for num in range(50,550,50)]

In [5]:
frames = []

for modelname in modelnames:
    
    for filename in filenames:
        
        filepath = os.path.join(input_dir, modelname, filename)
        
        df = pd.read_csv(filepath)
        
        df = df.loc[:,('smiles', 'bcl2', 'bclxl', 'bclw')]
        df.loc[:,'model'] = modelname
        df.loc[:,'checkpoint'] = filename.split('.')[-1]
        
        frames.append(df)

In [6]:
data = []

for df in tqdm.tqdm(frames):

    s_sr, s_nov, s_div = evaluate_sr_nov_div(df, referece_smiles_iter, 'bcl2_bclxl_bclw')
    
    data.append({'SR':s_sr, 'Nov':s_nov, 'Div':s_div, 'Model':df.loc[0,'model'], 'Ckpt':df.loc[0,'checkpoint']})

100%|██████████| 10/10 [00:02<00:00,  4.56it/s]


In [7]:
df_records = pd.DataFrame(data)
df_records.loc[:,'HMean'] = (df_records.loc[:,'SR'] * df_records.loc[:,'Nov'] * df_records.loc[:,'Div']) ** 0.333

df_records

,SR,Nov,Div,Model,Ckpt,HMean
0,0.011,1.0,0.660186,zinc15,0050,0.193969
1,0.016,1.0,0.487441,zinc15,0100,0.198632
2,0.472,1.0,0.593304,zinc15,0150,0.654522
3,0.090,1.0,0.630770,zinc15,0200,0.384698
4,0.080,1.0,0.630366,zinc15,0250,0.369823
5,0.127,1.0,0.606638,zinc15,0300,0.425875
6,0.030,1.0,0.821601,zinc15,0350,0.291383
7,0.036,1.0,0.824040,zinc15,0400,0.309927
8,0.034,1.0,0.840132,zinc15,0450,0.306048
9,0.067,1.0,0.763474,zinc15,0500,0.371581
